In [2]:
import pandas as pd
import bz2
import xml.etree.ElementTree as ET
import re
import os
import glob
from pathlib import Path

pd.set_option('display.max_colwidth', None)

In [3]:
BASE_DIR = Path("")  # your project's directory
DOT_DIR = BASE_DIR / "dot_files"  # should contain the three datasets
